In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

from app.database.connection import SessionLocal
db = SessionLocal()


from app.ml.data_loader import load_inventory_dataset
df = load_inventory_dataset(db)

# -----------------------------
# 1️⃣ Check dataset existence
# -----------------------------
if df is None or df.empty:
    print("⛔ Dataset is empty. Cannot evaluate model.")
else:

    # -----------------------------
    # 2️⃣ Check required columns
    # -----------------------------
    required_columns = [
        "quantity",
        "old_quantity",
        "new_quantity",
        "pharmacy_id",
        "medication_id"
    ]

    missing = [col for col in required_columns if col not in df.columns]

    if missing:
        print(f"⛔ Missing required columns: {missing}")
    else:

        SHORTAGE_THRESHOLD = 10
        df["shortage"] = (df["quantity"] <= SHORTAGE_THRESHOLD).astype(int)

        # --- Feature engineering (YOUR ORIGINAL CODE) ---
        df["stock_change"] = df["new_quantity"] - df["old_quantity"]
        df["is_low_stock"] = (df["quantity"] < 20).astype(int)
        df["medication_freq"] = df.groupby("medication_id")["medication_id"].transform("count")

        features = [
            "quantity",
            "stock_change",
            "is_low_stock",
            "medication_freq",
            "pharmacy_id",
            "medication_id"
        ]

        target = "shortage"

        X = df[features]
        y = df[target]

        # -----------------------------
        # 3️⃣ Check minimum samples
        # -----------------------------
        if len(X) < 5:
            print("⛔ Not enough samples to evaluate model.")
        else:

            model_path = "app/ml/shortage_model.joblib"

            # -----------------------------
            # 4️⃣ Check model existence
            # -----------------------------
            if not os.path.exists(model_path):
                print("⛔ Model file not found. Train model first.")
            else:
                model = joblib.load(model_path)

                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.2, random_state=42
                )

                y_pred = model.predict(X_test)

                print("Accuracy:", accuracy_score(y_test, y_pred))
                print("\nClassification Report:")
                print(classification_report(y_test, y_pred))
                print("\nConfusion Matrix:")
                print(confusion_matrix(y_test, y_pred))

Not enough inventory rows (5), creating dataset from stock history
Accuracy: 1.0

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1


Confusion Matrix:
[[1]]


C:\Users\UserPc\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:620: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [2]:
print(df.columns)
print(df.head())

Index(['pharmacy_id', 'medication_id', 'quantity', 'old_quantity',
       'new_quantity', 'changed_at', 'shortage', 'stock_change',
       'is_low_stock', 'medication_freq'],
      dtype='str')
   pharmacy_id  medication_id  quantity  old_quantity  new_quantity  \
0            1              1         5            20             5   
1            1              2        30            40            30   
2            1              3       100             0           100   
3            2              1         8            15             8   
4            2              3        50            55            50   

                  changed_at  shortage  stock_change  is_low_stock  \
0 2026-02-09 21:55:19.647410         1           -15             1   
1 2026-02-09 21:55:19.657839         0           -10             0   
2 2026-02-10 11:28:16.580215         0           100             0   
3 2026-02-09 21:55:19.657896         1            -7             1   
4 2026-02-09 21:55:19.657918 